<a href="https://colab.research.google.com/github/cmcnicci/TokenManager/blob/main/query_expansion_kg/manually_entity_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install tensorflow
!pip install qwikidata
!pip install pywikibot
!pip install mwparserfromhell
!pip install --upgrade farm-haystack
!pip install haystack-ai
!pip install haystack-ai[faiss]
!pip show haystack-ai

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-y_hv66ja
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-y_hv66ja
  Resolved https://github.com/deepset-ai/haystack.git to commit b12af1e6a987cde382abd03cfcd6b34951ca722f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Name: haystack-ai
Version: 2.12.0rc0
Summary: LLM framework to build customizable, production-ready LLM applications. Connect components (models, vector DBs, file converters) to pipelines or agents that can interact with your data.
Home-page: https://github.com/deepset-ai/haystack
Author: 
Author-email: "deepset.ai" <malte.pietsch@deepset.ai>
License-Expression: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: haystack-experimental, jinja2, jsonschema, lazy-imports, more-itertools, networkx, numpy, openai, posthog, pydanti

In [7]:
import pandas as pd
from typing import List
import requests
import pandas as pd
from haystack import Document
import pandas as pd
from haystack import Document
from haystack_integrations.document_stores.faiss import FAISSDocumentStore

# Инициализация FAISSDocumentStore
document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    embedding_dim=768,  # Размерность эмбеддингов для "mpnet-base"
    return_embedding=True
)

# Удаление существующих документов
document_store.delete_documents()
from haystack.nodes import RAGenerator, DensePassageRetriever, EmbeddingRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents
from haystack.pipelines import DocumentSearchPipeline
from haystack.document_stores import InMemoryDocumentStore

from datasets import load_dataset

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

import nltk.data
nltk.download('punkt')

ModuleNotFoundError: No module named 'haystack_integrations'

In [ ]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Delete existing documents in documents store
document_store.delete_documents()

In [ ]:
vali_wikiqa_data = load_dataset(
    'wiki_qa',
    split='train',
)
pd_test_data = pd.DataFrame(vali_wikiqa_data)

pd_test_data = pd.DataFrame(vali_wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_test_data.shape[0]):
  last_char = pd_test_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_test_data['answer'][index] = pd_test_data['answer'][index] + '.'


processed_test_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])
processed_all_wikiqa = pd.DataFrame(columns = ["question_id", "document_title", "document_content"])

for question_id in pd_test_data['question_id'].unique().tolist():
    # print(question_id)
    pd_question = pd_test_data.loc[pd_test_data['question_id'] == question_id]
    # print(pd_question)
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    # print(pd_question["document_title"].iloc[0])
    processed_all_wikiqa = processed_all_wikiqa.append({"question_id": question_id,
                                                        "document_title": pd_question["document_title"].iloc[0],
                                                        "document_content" : content}, ignore_index=True)
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0],
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        processed_test_wikiqa = processed_test_wikiqa.append(new_row, ignore_index=True)
processed_test_wikiqa

In [ ]:
docs = []
for index, row in processed_test_wikiqa.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store.write_documents(docs)
document_store.get_document_count()

In [ ]:
vali_wikiqa_data = load_dataset(
    'wiki_qa',
    split='test',
)
pd_test_data = pd.DataFrame(vali_wikiqa_data)

pd_test_data = pd.DataFrame(vali_wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_test_data.shape[0]):
  last_char = pd_test_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_test_data['answer'][index] = pd_test_data['answer'][index] + '.'


processed_test_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])
processed_all_wikiqa = pd.DataFrame(columns = ["question_id", "document_title", "document_content"])

for question_id in pd_test_data['question_id'].unique().tolist():
    # print(question_id)
    pd_question = pd_test_data.loc[pd_test_data['question_id'] == question_id]
    # print(pd_question)
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    # print(pd_question["document_title"].iloc[0])
    processed_all_wikiqa = processed_all_wikiqa.append({"question_id": question_id,
                                                        "document_title": pd_question["document_title"].iloc[0],
                                                        "document_content" : content}, ignore_index=True)
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0],
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        processed_test_wikiqa = processed_test_wikiqa.append(new_row, ignore_index=True)
processed_test_wikiqa

In [ ]:
docs = []
for index, row in processed_test_wikiqa.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store.write_documents(docs)
document_store.get_document_count()


In [ ]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers"
)

document_store.update_embeddings(retriever, batch_size=128)

In [ ]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever = retriever)
document_store.get_embedding_count()

In [ ]:
import pandas as pd

pd_test_data = pd.read_csv("/content/entity&relationship_extraction.csv")

In [ ]:
#df_direct_answer = pd_test_data[pd_test_data['answer by entities and relationships'].notna()]
cols = ['answer by entities and relationships', 'entities']
df_direct_answer = pd_test_data[pd_test_data[cols].notnull().any(axis=1)]
df_direct_answer

# query expansion using knowledge graph

In [ ]:
def query_expansion(original_query):
  pd_question = df_direct_answer.loc[df_direct_answer['question'] == original_query]

  if not pd_question.empty:

    # first
    if str(pd_question['answer by entities and relationships'].iloc[0]) != 'nan':
      new_query = original_query + " " + pd_question['answer by entities and relationships'].iloc[0]
      return new_query

    # second
    elif str(pd_question['entities'].iloc[0]) != 'nan' and len(pd_question['entities'].iloc[0].split(", ")) == 1:
      # entity_id_list = pd_question['entities'].iloc[0].split(", ")
      entity_id = pd_question['entities'].iloc[0]
      entity_dict = get_entity_dict_from_api(entity_id)
      entity_dict = WikidataItem(entity_dict)
      description = entity_dict.get_description()
      new_query = original_query + " " + description
      return new_query

    else:
      if str(pd_question['entities_name'].iloc[0]) != 'nan':
        new_query = original_query + str(pd_question['entities_name'].iloc[0])
        return new_query

  return original_query

In [ ]:
import json
import requests

MY_GCUBE_TOKEN = "f83a241f-c321-4473-ae3c-0dd2151382fd-843339462"

class WATAnnotation:
    # An entity annotated by WAT

    def __init__(self, d):

        # char offset (included)
        self.start = d['start']
        # char offset (not included)
        self.end = d['end']

        # annotation accuracy
        self.rho = d['rho']
        # spot-entity probability
        self.prior_prob = d['explanation']['prior_explanation']['entity_mention_probability']

        # annotated text
        self.spot = d['spot']

        # Wikpedia entity info
        self.wiki_id = d['id']
        self.wiki_title = d['title']


    def json_dict(self):
        # Simple dictionary representation
        return {'wiki_title': self.wiki_title,
                'wiki_id': self.wiki_id,
                'start': self.start,
                'end': self.end,
                'rho': self.rho,
                'prior_prob': self.prior_prob
                }


def wat_entity_linking(text):
    # Main method, text annotation with WAT entity linking system
    wat_url = 'https://wat.d4science.org/wat/tag/tag'
    payload = [("gcube-token", MY_GCUBE_TOKEN),
               ("text", text),
               ("lang", 'en'),
               ("tokenizer", "nlp4j"),
               ('debug', 9),
               ("method",
                "spotter:includeUserHint=true:includeNamedEntity=true:includeNounPhrase=true,prior:k=50,filter-valid,centroid:rescore=true,topk:k=5,voting:relatedness=lm,ranker:model=0046.model,confidence:model=pruner-wiki.linear")]

    response = requests.get(wat_url, params=payload)
    return [WATAnnotation(a) for a in response.json()['annotations']]

def print_wat_annotations(wat_annotations):
    json_list = [w.json_dict() for w in wat_annotations]
    print(json.dumps(json_list, indent=4))

In [ ]:
import urllib
import json
import requests
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

def query_to_description(original_question):
    wat_annotations = wat_entity_linking(original_question)
    json_list = [w.json_dict() for w in wat_annotations]
    print(json_list)
    if json_list != []:
      if len(json_list) == 1:
          if "wiki_title" in json_list[0].keys():
              wiki_title = json_list[0]["wiki_title"]
              url = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles=" + wiki_title + "&format=json"
              #url = "https://en.wikipedia.org/w/api.php?action=query&prop=wikitext&ppprop=wikibase_item&redirects=1&titles=" + wiki_title
              print(url)
              r = requests.get(url)
              data = json.loads(r.text)
              data = json.dumps(data)
              # extract wikidata entity id
              data = data.partition("wikibase_item\": \"")[2]
              entity_id = data.partition("\"")[0]
              print("entity id: ", entity_id)
              entity_dict = get_entity_dict_from_api(entity_id)
              entity_dict = WikidataItem(entity_dict)
              description = entity_dict.get_description()
              if description != None:
                  print("Description: ", description)
                  return description
    return None

# Accuracy with original query

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0
number_correctly_retrieved_passage = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  #if query != new_query:
  number_total += 1
  print("Original Query: ", query)

  result = pipeline.run(
            query=query,
            params={"Retriever": {"top_k": 1}}
            )

  retrieved_document = result["documents"][0].content
  target_document = processed_test_wikiqa['document_content'][index]
  # print("query: ", query)

  if retrieved_document == target_document:
      number_correctly_retrieved_passage += 1


      sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])

      sentence_document_store = InMemoryDocumentStore()
      sentence_document_store.delete_documents()

      docs = []
      for sentence in sentence_list:
        doc = Document(content=sentence,
                      meta={"article_title": row['document_title']})
        docs.append(doc)
      sentence_document_store.write_documents(docs)

      sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")

      sentence_document_store.update_embeddings(sentence_retriever)
      sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
      sentence_result = sentence_pipeline.run(query=query,
                                              params={"Retriever": {"top_k": 1}})
      retrieved_sentence = sentence_result["documents"][0].content
      target_sentence = processed_test_wikiqa['answer'][index]
      if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
          number_correct += 1
          print("!!!!!!!!!")
          print("Found")
      else:
          print("Not Found")

In [ ]:
print(number_total)
print(number_correctly_retrieved_passage / number_total)
print(number_correct / number_correctly_retrieved_passage)

# Caculate accuracy with query expansion (WAT)

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0
number_correctly_retrieved_passage = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  #if query != new_query:
  number_total += 1

  entity_description = query_to_description(query)
  if entity_description != None and entity_description != "":
      # query = entity_description
      new_query = query + " " + entity_description
      print("query: ", query)
  else:
    new_query = query


  result = pipeline.run(
            query=new_query,
            params={"Retriever": {"top_k": 1}}
            )

  retrieved_document = result["documents"][0].content
  target_document = processed_test_wikiqa['document_content'][index]
  # print("query: ", query)

  if retrieved_document == target_document:
      number_correctly_retrieved_passage += 1


      sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])

      sentence_document_store = InMemoryDocumentStore()
      sentence_document_store.delete_documents()

      docs = []
      for sentence in sentence_list:
        doc = Document(content=sentence,
                      meta={"article_title": row['document_title']})
        docs.append(doc)
      sentence_document_store.write_documents(docs)

      sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")

      sentence_document_store.update_embeddings(sentence_retriever)
      sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
      sentence_result = sentence_pipeline.run(query=new_query,
                                              params={"Retriever": {"top_k": 1}})
      retrieved_sentence = sentence_result["documents"][0].content
      target_sentence = processed_test_wikiqa['answer'][index]
      if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
          number_correct += 1
          print("!!!!!!!!!")
          print("Found")
      else:
          print("Not Found")

In [ ]:
print(number_total)
print(number_correctly_retrieved_passage / number_total)
print(number_correct / number_correctly_retrieved_passage)

# Calculate accuracy with query expansion （labeling manually）

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0
number_correctly_retrieved_passage = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  new_query = query_expansion(query)
  #if query != new_query:
  number_total += 1
  print("Original Query: ", query)
  print("New Query: ", new_query)

  result = pipeline.run(
            query=new_query,
            params={"Retriever": {"top_k": 1}}
            )

  retrieved_document = result["documents"][0].content
  target_document = processed_test_wikiqa['document_content'][index]
  # print("query: ", query)

  if retrieved_document == target_document:
      number_correctly_retrieved_passage += 1
      sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])

      sentence_document_store = InMemoryDocumentStore()
      sentence_document_store.delete_documents()

      docs = []
      for sentence in sentence_list:
        doc = Document(content=sentence,
                      meta={"article_title": row['document_title']})
        docs.append(doc)
      sentence_document_store.write_documents(docs)

      sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")

      sentence_document_store.update_embeddings(sentence_retriever)
      sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
      sentence_result = sentence_pipeline.run(query=new_query,
                                              params={"Retriever": {"top_k": 1}})
      retrieved_sentence = sentence_result["documents"][0].content
      target_sentence = processed_test_wikiqa['answer'][index]
      if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
          number_correct += 1
          print("!!!!!!!!!")
          print("Found")
      else:
          print("Not Found")

In [ ]:
print(number_total)
print(number_correctly_retrieved_passage / number_total)
print(number_correct / number_correctly_retrieved_passage)